## Deploy Model to AKS Cluster ##

><b>Pre-requisites</b>
>
> Azure ML SDK has been installed.
>
> You have run <b>jupyter notebook</b> from a command line that was based in your project directory.
>
> You have successfully executed <b>az login</b> from said command line.

<b>NOTE</b> Deploying a cluster and publishing a service can take upwards of 30 minutes. Further, the subscription you are working with for this lab may not be suitable for each of you to create an AKS cluster due to core limitations. Work on the cells up to but not including Provision the AKS cluster

This notebook registers the model with the AML service, builds a docker container which is stored in an Azure Container Registry, deploys an AKS cluster, and finally tests the service in place. 


## Imports and Variables ##

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image
from azureml.core.model import Model
from azureml.core.workspace import Workspace

#from azure.storage.blob import BlockBlobService
#from azure.storage.blob import ContentSettings

import azureml.core
print(azureml.core.VERSION)

#AZURE_STORAGE_ACCOUNT_NAME = "catjulydemodata"
#AZURE_STORAGE_ACCOUNT_KEY = "94nA2VT9YR8cWuT0eqUMM8uWJiRgTUpkGwONKKD/WsZIPC8UDH6PBQFSgTTh01qbY9WqCF0HoWvOnb1wRmIM6Q=="
#AZURE_STORAGE_MODEL_CONTAINER_NAME = "factorymodel"
#AZURE_STORAGE_MODEL_BLOB_NAME = "factorymodel.pkl"

#LOCAL_SYSTEM_DATA_DIRECTORY = "./TrainingData"
#LOCAL_SYSTEM_MODEL_FILE = "{}/{}".format(LOCAL_SYSTEM_DATA_DIRECTORY, AZURE_STORAGE_MODEL_BLOB_NAME)

ML_MODEL_NAME = "factorymodel.pkl"
LOCAL_SYSTEM_MODEL_FILE = "./{}".format(ML_MODEL_NAME)

MODEL_TAGS = ["factory", "predictivemaintenance"]
MODEL_DESCRIPTION = "Predict factory floor failures."

print(LOCAL_SYSTEM_MODEL_FILE)

# Get workspace
Load existing workspace from the config file info.

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Register the model ##

Register an existing trained model, add descirption and tags.

In [ ]:
%%time

print("Register file {} with name {}".format(LOCAL_SYSTEM_MODEL_FILE,ML_MODEL_NAME))
model = Model.register(model_path = LOCAL_SYSTEM_MODEL_FILE, # this points to a local file
                       model_name = ML_MODEL_NAME, # this is the name the model is registered as
                       tags = MODEL_TAGS,
                       description = MODEL_DESCRIPTION,
                       workspace = ws)

print(model.name, model.description, model.version)

# Create an image
Create an image (docker container) using the registered model the script that will load and run the model.

In [ ]:
!more score.py

In [ ]:
!more amlenv.yml

This next step creates the actual container in the ACR for this project.

In [ ]:
image = Image.create(name = "factoryimage22",
                     models = [model],
                     runtime = "python",
                     execution_script = "score.py",
                     conda_file = "amlenv.yml",
                     tags = MODEL_TAGS,
                     description = MODEL_DESCRIPTION,
                     workspace = ws)

In [ ]:
%%time
image.wait_for_creation(show_output = True)

# Provision the AKS Cluster

><b>NOTE</b> These steps take a LONG time and should not be run during this lab. The section is here to complete the >flow from creation to operationalizing a model.

This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.

## Connect existing cluster ##

Use this cell if you already have your cluster up and running, otherwise skip it and go to the next cell to create one.

In [ ]:
%%time
create_name = 'aml-aks-8' 
resource_id ='/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/dangamlsdk/providers/Microsoft.ContainerService/managedClusters/aml-aks-8b2e247882f70b30'

print(ws.name, create_name, resource_id)

# Create the cluster
aks_target = AksCompute.attach(workspace=ws, name=create_name, resource_id=resource_id)
# Wait for the operation to complete
aks_target.wait_for_provisioning(True)


## Create Cluster ##

Use this cell to create a new AKS cluster, otherwise if you already have one, use the cell above to connect to an existing cluster.

In [ ]:
%%time

# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'aml-aks-d' 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

In [ ]:
%%time
aks_target.wait_for_provisioning(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

## Deploy Web Service to AKS Cluster ##

In [ ]:
#Set the web service configuration (using default here)
aks_config = AksWebservice.deploy_configuration()

In [ ]:
%%time
aks_service_name ='aks-service-1'

aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)
print(aks_service.error)
print(aks_service.get_logs() )

## Test the service ##

In [ ]:
%%time
import json

test_sample = json.dumps({'input_df':[{'id':1,'volt':241.0,'rotate':120.0,'temp':189.0,'time':3}]})
test_sample = bytes(test_sample,encoding = 'utf8')

prediction = aks_service.run(input_data = test_sample)
print(prediction)

## Clean up ##
Delete the service, image and model.

In [ ]:
%%time
aks_service.delete()
image.delete()
model.delete()